# Task 3: A/B Hypothesis Testing

## Hypotheses to test:
1. There are no risk differences across provinces
2. There are no risk differences between zipcodes
3. There is no significant margin (profit) difference between zip codes
4. There is no significant risk difference between Women and men

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Load the data (use df_valid from previous analysis or reload)
# Let's reload to be sure we have clean data
df = pd.read_csv('../data/raw/insurance_data.txt', delimiter='|', low_memory=False)

# Convert key columns
df['TotalPremium'] = pd.to_numeric(df['TotalPremium'], errors='coerce')
df['TotalClaims'] = pd.to_numeric(df['TotalClaims'], errors='coerce')
df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')

# Filter out zero premiums for accurate analysis
df_valid = df[df['TotalPremium'] > 0].copy()
df_valid['LossRatio'] = df_valid['TotalClaims'] / df_valid['TotalPremium']

print(f"Data loaded. Shape: {df_valid.shape}")
print(f"Valid policies (premium > 0): {len(df_valid):,}")

Data loaded. Shape: (618176, 53)
Valid policies (premium > 0): 618,176
